In [ ]:
import pyodbc
from import_function import import_scoring_data, import_data
from features_by_customer_type_3 import customer_type_features
from care_features_2 import care_features
# from features_by_customer_type import customer_type_features
conn = pyodbc.connect(dsn='VerticaProd')
ct='New'
response=['ABANDONED']
#  AND CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX'
care_cont_features, care_bool_features, care_catag_features, care_cont_score_features, care_bool_score_features, care_catag_score_features=care_features(ct)
cont_features,bool_features,catag_features,cont_score_features,bool_score_features,catag_score_features=customer_type_features(ct)
data = "SELECT A.AUTH_ID,ABANDONED, AGE_TAXPAYER, AGI,	AMOUNT_INCOME_TAX_WITHHELD,	AMOUNT_INCOME_TAX,	AMOUNT_REFUND,	AMOUNT_SALARIES_AND_WAGES,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	AMOUNT_TAX,	AMOUNT_TOTAL_DEDUCTIONS,	AMOUNT_TOTAL_PAYMENTS,	AMOUNT_TOTAL_TAX,	COMPLETED_SKU,	COST_PER_CUST_LAG7,	COST_PER_CUST,	CUSTOMER_TYPE,	DMA_AREA,	FED_FORM_TYPE,	FILING_STATUS,	FIRST_COMPLETE_APP_TYPE,	FIRST_COMPLETE_DEVICE_TYPE,	FSCHA_FLAG,	IMPORT_TYPE,	NEAUTH_DEVICE_TYPE,	NUM_DEPENDENTS,	NUM_EXEMPTIONS,	NUM_W2,	PRS_SCORE,	REFUND_TRANSFER_FLAG,	REJECT_COUNT,	REQUIRED_TAKE_FLAG,	SESSIONS_TO_COMPLETE,	START_DEVICE_TYPE,	STATE_ATTACH_COUNT,	STUDENT_TAXPAYER,	TOTAL_REVENUE,	VAUTH_DEVICE_TYPE FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SAMPLE B ON A.CUSTOMER_KEY=B.CUSTOMER_KEY WHERE TAX_YEAR<2016 "
care_data="SELECT DISTINCT  A.AUTH_ID,  TAX_YEAR, AXC_CARE,ENTERING_CONTACT_US_EXPERIENCE,TOTAL_AGENT_INTERACTION_SECONDS,CARE_REFERRER,	FIRST_HS_INTENT,	FIRST_SEARCH_MANUAL_LOCATION_DETAIL FROM CTG_ANALYTICS_WS.SM_CARE_DATA_MINING A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE A.TAX_YEAR=2015 "
df_no_pca= import_data(data,care_data, cont_features, bool_features, catag_features,care_cont_features, care_bool_features,care_catag_features)


In [ ]:
from bin_cont_features import bin_pca
j='True'
response='ABANDONED'

print df_no_pca.dtypes
dummy_pca, length_dict = bin_pca(df_no_pca,response, j)
scoring_list=list(dummy_pca)

In [ ]:
response='ABANDONED'
ct='New'
from feature_clustering import feature_clustering
i='pca'

df_no_pca,y, plsca= feature_clustering(dummy_pca,response, i)
print list(df_no_pca)




In [ ]:
from clustering import obs_clustering
clust='False'
df=df_no_pca
print df
df, df_w_labels, df_w_clust=obs_clustering(df,response,clust)
print df

In [ ]:
from get_arrays import get_arrays
i='False'
j='False'
dummy_pca=df
response = 'ABANDONED'
x, y = get_arrays(dummy_pca, response, i, j)
print x.shape
from sklearn.feature_selection import SelectPercentile, f_classif
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
selector = SelectPercentile(f_classif, percentile=20)
selector.fit(x, y)
print selector.pvalues_
df_pvalues=pd.DataFrame(selector.pvalues_,columns=['p_value'], index=list(x))
print df_pvalues
sig_features=df_pvalues[df_pvalues['p_value'] <=0.2]
sig_features.reset_index(['index'], inplace=True)
sig_features.drop(['p_value'], axis=1, inplace=True)
print sig_features


x=x[sig_features['index'].tolist()]
print x.shape



In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestClassifier
import numpy as np

train_sizes, train_scores, valid_scores = learning_curve(RandomForestClassifier(criterion='entropy', n_estimators=20, random_state=np.random.RandomState(0)), x, y,train_sizes=[0.001, 0.01, 0.1, 0.5],  scoring='roc_auc', cv=5)
print train_sizes
for i in train_scores:
    print np.mean(i)
for K in valid_scores:
    print np.mean(K)

In [ ]:
from model_algorithm import algorithm
response = 'ABANDONED'
models, name = algorithm(x, y, response)

In [ ]:
import pandas as pd
import numpy as np
import pickle
ct='New'
response=['ABANDONED']
filename = 'finalized_model.sav'
filename2 ='name.sav'
filename3 = 'bin_model.sav'
filename4 = 'feature_selection.sav'

filename5 ='score_model.sav'
from care_features_2 import care_features
from predicting_code import transform_to_pca, bin_pca_score_set, get_scoring_arrays, score_set_feature_selection, predict,get_scoring_arrays
from import_function import import_scoring_data
from features_by_customer_type_3 import customer_type_features
defection=pd.DataFrame()
care_cont_features, care_bool_features, care_catag_features, care_cont_score_features, care_bool_score_features, care_catag_score_features=care_features(ct)
cont_features,bool_features,catag_features,cont_score_features,bool_score_features,catag_score_features=customer_type_features(ct)

# load the model from disk
models = pickle.load(open(filename, 'rb'))
name = pickle.load(open(filename2, 'rb'))
length_dict = pickle.load(open(filename3, 'rb'))
plsca = pickle.load(open(filename4, 'rb'))


scoring_data = "SELECT  A.AUTH_ID, AGE_TAXPAYER,	AGI,	AMOUNT_INCOME_TAX_WITHHELD,	AMOUNT_INCOME_TAX,	AMOUNT_REFUND,	AMOUNT_SALARIES_AND_WAGES,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	AMOUNT_TAX,	AMOUNT_TOTAL_DEDUCTIONS,	AMOUNT_TOTAL_PAYMENTS,	AMOUNT_TOTAL_TAX,	CASE WHEN COMPLETED_SKU in ('850|Paid Self Employed','900|Paid Home and Business') then '900|Paid Home and Business' else COMPLETED_SKU end as COMPLETED_SKU ,	COST_PER_CUST_LAG7,	COST_PER_CUST,	CUSTOMER_TYPE,	DMA_AREA,	FED_FORM_TYPE,	CASE WHEN FILING_STATUS='MarringFilingSeparately' THEN 'MarriedFilingSeparately' ELSE FILING_STATUS END AS FILING_STATUS  ,	FIRST_COMPLETE_APP_TYPE,	FIRST_COMPLETE_DEVICE_TYPE,	FSCHA_FLAG,	IMPORT_TYPE,	NEAUTH_DEVICE_TYPE,	NUM_DEPENDENTS,	NUM_EXEMPTIONS,	NUM_W2,	PRS_SCORE,	REFUND_TRANSFER_FLAG,	REJECT_COUNT,	REQUIRED_TAKE_FLAG,	SESSIONS_TO_COMPLETE,	START_DEVICE_TYPE,	STATE_ATTACH_COUNT,	STUDENT_TAXPAYER,	TOTAL_REVENUE,	VAUTH_DEVICE_TYPE FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SCORE_SAMPLE B ON A.CUSTOMER_KEY=B.CUSTOMER_KEY WHERE TAX_YEAR=2016"
care_score_data="SELECT DISTINCT  A.AUTH_ID, TAX_YEAR, AXC_CARE,ENTERING_CONTACT_US_EXPERIENCE,TOTAL_AGENT_INTERACTION_SECONDS,CARE_REFERRER,	FIRST_HS_INTENT,	FIRST_SEARCH_MANUAL_LOCATION_DETAIL FROM CTG_ANALYTICS_WS.SM_CARE_DATA_MINING A INNER JOIN CTG_ANALYTICS_WS.SM_CUSTOMER_RANDOM_SCORE_SAMPLE B ON A.AUTH_ID=B.AUTH_ID WHERE A.TAX_YEAR=2016 "

scoring_df=import_scoring_data(scoring_data,care_score_data, cont_score_features, bool_score_features, catag_score_features, care_cont_score_features, care_bool_score_features, care_catag_score_features)

i, j, k=['False','True','pca' ]
print scoring_df
scoring_df.reset_index(['AUTH_ID'], inplace=True)

index=scoring_df['AUTH_ID']
print scoring_df.head()

# scoring_df.drop(['AUTH_ID'], axis=1, inplace=True)
# new_list2=list(set(list(df_no_pca)) & set(list(scoring_df)))
# df_scoring=df_no_pca[new_list2]
# scoring_df.reset_index(['CUSTOMER_KEY'], inplace=True)

# scoring_df.drop(['CUSTOMER_KEY'], axis=1, inplace=True)
# x=scoring_df[scoring_list]
# df_trans_pca = transform_to_pca(scoring_df, fitted_pca, i)
# print df_trans_pca
print length_dict
scoring_df_trans = bin_pca_score_set(scoring_df, length_dict, j)

print scoring_df_trans.shape, x.shape
a=list(x)
b=list(scoring_df_trans)
final=list(set(b).difference(a))
scoring_df_trans.drop(final, axis=1, inplace=True)
print scoring_df_trans.shape
x.drop(list(scoring_df_trans), axis=1, inplace=True)
print x.shape
c=list(x)

m=scoring_df_trans.shape[0]     
n= x.shape[1]
print  scoring_df_trans.shape, x.shape

zeros_df=pd.DataFrame(np.zeros((m, n)), columns=c).astype('bool')



# scoring_df_trans.reset_index(['CUSTOMER_KEY'], inplace=True)
print scoring_df_trans

final_scoring_df=pd.concat([zeros_df,scoring_df_trans,index ], axis=1)
final_scoring_df.set_index('AUTH_ID', inplace=True)
print final_scoring_df.shape

scoring_df_trans=final_scoring_df
print scoring_df_trans.shape
x_score, index=score_set_feature_selection(scoring_df_trans,plsca, k)

# x_score.set_index('CUSTOMER_KEY', inplace=True)
print list(x_score)
x_score.drop(['AUTH_ID'], axis=1, inplace=True)
df_p=predict(models,name,x_score, index)
defection=pd.concat([df_p, defection], axis=1)
print defection.head()
defection.to_csv(path_or_buf='defection_model_1.txt', index=True)


In [ ]:
import pandas as pd
from data_intersection import scoring_data_intersection
y_df=df_no_pca[response]
print scoring_df.head()

data, _ = scoring_data_intersection(df_no_pca, scoring_df)
print data.shape
print scoring_df.head()
df_pca=pd.concat([y_df, data], axis=1)
# df_pca.reset_index(['CUSTOMER_KEY'], inplace=True)
print df_pca.shape
